In [3]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Estimator, Sampler
from qiskit_aer.noise import NoiseModel

from squlearn import Executor
from squlearn.encoding_circuit import QiskitEncodingCircuit
from squlearn.observables import SinglePauli
from squlearn.qnn import QNNRegressor, SquaredLoss
from squlearn.optimizers import Adam, SLSQP, SGLBO

import matplotlib.pyplot as plt
import matplotlib as mpl

In [4]:
nqubits = 4

qc = QuantumCircuit(nqubits)
p = ParameterVector('p', 8)
x = ParameterVector('x', 1)

p2 = np.array([-0.13005136, -1.34331017, 2.44392299, 1.64405423, -0.36315523, 0.45344925,
 0.18577077, -0.13904446])

for i in range(nqubits):
 qc.rx(1*np.arccos(x[0]), i)
 qc.ry(p2[i], i)

qc.cx(0, 1)
qc.cx(2, 3)
qc.cx(1, 2)
qc.cx(3, 0)

for i in range(nqubits):
 qc.rx(1*np.arccos(x[0]), i)
 qc.ry(p2[i+nqubits], i)

pqc = QiskitEncodingCircuit(qc)
pqc.draw()

┌────────────────┐┌──────────────┐          ┌───┐┌────────────────┐»
q_0: ┤ Rx(acos(x[0])) ├┤ Ry(-0.13005) ├──■───────┤ X ├┤ Rx(acos(x[0])) ├»
     ├────────────────┤├─────────────┬┘┌─┴─┐     └─┬─┘├────────────────┤»
q_1: ┤ Rx(acos(x[0])) ├┤ Ry(-1.3433) ├─┤ X ├──■────┼──┤ Rx(acos(x[0])) ├»
     ├────────────────┤└┬────────────┤ └───┘┌─┴─┐  │  ├────────────────┤»
q_2: ┤ Rx(acos(x[0])) ├─┤ Ry(2.4439) ├───■──┤ X ├──┼──┤ Rx(acos(x[0])) ├»
     ├────────────────┤ ├────────────┤ ┌─┴─┐└───┘  │  ├────────────────┤»
q_3: ┤ Rx(acos(x[0])) ├─┤ Ry(1.6441) ├─┤ X ├───────■──┤ Rx(acos(x[0])) ├»
     └────────────────┘ └────────────┘ └───┘          └────────────────┘»
«     ┌──────────────┐
«q_0: ┤ Ry(-0.36316) ├
«     ├─────────────┬┘
«q_1: ┤ Ry(0.45345) ├─
«     ├─────────────┤ 
«q_2: ┤ Ry(0.18577) ├─
«     ├─────────────┴┐
«q_3: ┤ Ry(-0.13904) ├
«     └──────────────┘

In [5]:
op = SinglePauli(nqubits,qubit=0,parameterized=True)
print(op)

SparsePauliOp(['IIIZ'],
              coeffs=[ParameterExpression(1.0*p[0])])


In [6]:
op.num_parameters

1

In [7]:
# Randomly initialize parameters of the encoding circuit
np.random.seed(13) #13
param_ini =  np.random.rand(pqc.num_parameters)
# Initialize parameters of the observable as ones
param_op_ini = np.ones(op.num_parameters)
x0 = [[i * 0.2] for i in range(15)]
optimizer_options = {"bo_aqc_func": "EI", "bo_aqc_optimizer": "lbfgs", "bo_bounds": [(0.0, 5.0)], "log_file": "sglbo_noiseless.log",
                     "bo_n_calls": 30, "bo_x0_points": x0, "maxiter": 100}

qnn_simulator = QNNRegressor(
    pqc,
    op,
    Executor("statevector_simulator"),
    SquaredLoss(),
    SGLBO(optimizer_options),
    #SLSQP(),
    #Adam({"lr": 0.2,"log_file": "adam_noiseless.log"}),
    param_ini,
    param_op_ini=param_op_ini,
    opt_param_op=True, # Keine Observablen optimierung
    parameter_seed=124
)

In [8]:
print(param_ini)
print(param_op_ini)

[]
[1.]


In [9]:
# Data that is inputted to the QNN
x_train = np.arange(-0.5, 0.6, 0.1)
# Data that is fitted by the QNN
y_train = np.sin(6.0*x_train)

In [10]:
def evaluate_loss(theta, qnn_regressor: QNNRegressor, X, y):
    param = theta[: qnn_regressor._qnn.num_parameters]
    param_op = theta[qnn_regressor._qnn.num_parameters :]

    loss_values = qnn_regressor._qnn.evaluate(qnn_regressor.loss.loss_args_tuple, X, param, param_op)

    loss_value = qnn_regressor.loss.value(
        loss_values,
        ground_truth=y,
        weights=np.ones(len(y)),
        iteration=0,
    )

    return loss_value




In [11]:
qnn_simulator.fit(x_train, y_train)

fit:   0%|          | 0/100 [00:00<?, ?it/s]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
qnn_simulator.param

In [ ]:
qnn_simulator.param_op

In [ ]:
evaluate_loss([qnn_simulator.param,qnn_simulator.param_op], qnn_simulator, x_train, y_train)

In [ ]:
x = np.arange(np.min(x_train), np.max(x_train) + 0.01, 0.01)
y = qnn_simulator.predict(x)  # function for evaluating the prediction of the QNN
plt.plot(x, np.sin(6.0*x), label="Parabola function")
plt.plot(x, y, label="QNN inference")
plt.plot(x_train, y_train, "x", label="training data")
plt.legend()
plt.title("QNN inference of a parabola function")
plt.show()

In [ ]:
# Lese die Optimierungskoordinaten aus der Datei "params.log"
optimization_data = np.loadtxt("sglbo_noiseless_params2.log")
optimization_x = optimization_data[:, 0]
optimization_y = optimization_data[:, 1]

# plot the decision function for each datapoint on the grid
xx, yy = np.meshgrid(np.linspace(-3, 3, 50), np.linspace(-1, 15, 50))
Z = []
for i in range(len(xx)):
    for j in range(len(xx[i])):
        param = np.array([xx[i][j], yy[i][j]])
        Z.append(evaluate_loss(param, qnn_simulator, x_train, y_train))

Z = np.array(Z)
Z = Z.reshape(xx.shape)
#norm = mpl.colors.Normalize(vmin=-max(abs(np.min(Z)),abs(np.max(Z))), vmax=max(abs(np.min(Z)),abs(np.max(Z))))
im = plt.imshow(
    Z,
    interpolation="bilinear",
    extent=(xx.min(), xx.max(), yy.min(), yy.max()),
    aspect="auto",
    origin="lower",
    cmap=plt.cm.PuOr_r,
    #norm=norm
)

# Verbinde die Punkte im Scatter Plot mit einer Linie
plt.plot(optimization_x, optimization_y, c='red', linestyle='-', linewidth=2, label='Convergence Line')

# Plot backtransformed hyperplane
contours = plt.contour(xx, yy, Z, levels=[0,0.5,1,2,3,4,5,6,7], linewidths=2, linestyles="dashed")

# Plot training data
plt.axis([-3, 3, -1, 15])
plt.xlabel('X-Axis')
plt.ylabel('Y-Axis')
plt.title('Contour Plot with Optimization Path')
plt.legend()
plt.show()